In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
#@title import packages
import keras
import numpy as np
import os
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import sys

!pip install rarfile segmentation-models git+https://github.com/davej23/image-segmentation-keras.git rioxarray
from rarfile import RarFile
from sklearn.metrics import *
import rioxarray as rxr

  Cloning https://github.com/davej23/image-segmentation-keras.git to /tmp/pip-req-build-irqe_u1_
  Running command git clone --filter=blob:none --quiet https://github.com/davej23/image-segmentation-keras.git /tmp/pip-req-build-irqe_u1_
  Resolved https://github.com/davej23/image-segmentation-keras.git to commit e01b0a8d5859854cd9d259a618829889166439f5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=34377 sha256=6ae58e789dd1df0baa6cd7c77748

In [2]:
base_dir = r"/content/gdrive/MyDrive/mudtrain/"
#@title Read training images and normalise
training_images_list = os.listdir(r"{}train/images/".format(base_dir))
training_masks_list = []
training_images = []
for n in training_images_list:
  training_masks_list.append(n)
  a = (np.array(rxr.open_rasterio(r"{}train/images/{}".format(base_dir,n))))
  a = (a-np.min(a)) / (np.max(a)-np.min(a))
  training_images.append(a)

## Training masks
training_masks = []
for n in training_masks_list:
  a = (np.array(rxr.open_rasterio(r"{}train/labels/{}".format(base_dir,n))))
  training_masks.append(a)


## Validation images
validation_images_list = os.listdir(r"{}val/images/".format(base_dir))
validation_masks_list = []
validation_images = []
for n in validation_images_list:
  validation_masks_list.append(n)
  a = (np.array(rxr.open_rasterio(r"{}val/images/{}".format(base_dir,n))))
  a = (a-np.min(a)) / (np.max(a)-np.min(a))
  validation_images.append(a)

## Validation masks
validation_masks = []
for n in validation_masks_list:
  a = (np.array(rxr.open_rasterio(r"{}val/labels/{}".format(base_dir,n))))
  validation_masks.append(a)

In [3]:
#@title Pre-process data, reshaping and transposing
for i in range(len(training_images)):
  training_images[i] = training_images[i].astype('float32')
  training_images[i] = training_images[i].T

for i in range(len(training_masks)):
  training_masks[i] = training_masks[i].reshape(1,256,256)
  training_masks[i] = training_masks[i].T

for i in range(len(validation_images)):
  validation_images[i] = validation_images[i].astype('float32')
  validation_images[i] = validation_images[i].T

for i in range(len(validation_masks)):
  validation_masks[i] = validation_masks[i].reshape(1,256,256)
  validation_masks[i] = validation_masks[i].T


for i in range(len(training_images)):
  training_images[i] = training_images[i].reshape(256,256,10)

for i in range(len(validation_images)):
  validation_images[i] = validation_images[i].reshape(256,256,10)


In [4]:
images=np.vstack([training_images])
val_images=np.vstack([validation_images])
images.shape

(889, 256, 256, 10)

In [5]:
masks=np.vstack([training_masks])
val_masks=np.vstack([validation_masks])
val_masks.shape

(223, 256, 256, 1)

In [6]:
import gc
del training_images,validation_images,training_masks,validation_masks,training_images_list,validation_images_list,
training_masks_list,validation_masks_list
gc.collect()

904

In [20]:
del images,masks,val_images,val_masks

In [7]:
#@title boundary loss
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models

#Shape of semantic segmentation mask
OUTPUT_SHAPE = (256, 256, 1)
def boundary_loss(y_true, y_pred):

    """
    Paper Implemented : https://arxiv.org/abs/1905.07852
    Using Binary Segmentation mask, generates boundary mask on fly and claculates boundary loss.
    :param y_true:
    :param y_pred:
    :return:
    """
    y_true=tf.cast(y_true,tf.float32)
    y_pred=tf.cast(y_pred,tf.float32)
    
    y_pred_bd = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
    y_true_bd = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
    y_pred_bd = y_pred_bd - (1 - y_pred)
    y_true_bd = y_true_bd - (1 - y_true)

    y_pred_bd_ext = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
    y_true_bd_ext = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
    y_pred_bd_ext = y_pred_bd_ext - (1 - y_pred)
    y_true_bd_ext = y_true_bd_ext - (1 - y_true)

    P = K.sum(y_pred_bd * y_true_bd_ext) / K.sum(y_pred_bd) + 1e-7
    R = K.sum(y_true_bd * y_pred_bd_ext) / K.sum(y_true_bd) + 1e-7
    F1_Score = 2 * P * R / (P + R + 1e-7)
    loss = K.mean(1 - F1_Score)
    
    return loss

In [8]:
from keras.callbacks import ModelCheckpoint, Callback
class AlphaScheduler(Callback):
  def init(self, alpha, update_fn):
    self.alpha = alpha
    self.update_fn = update_fn
  def on_epoch_end(self, epoch, logs=None):
    updated_alpha = self.update_fn(K.get_value(self.alpha))

alpha = K.variable(1, dtype='float32')

def update_alpha(value):
  return np.clip(value - 0.005, 0.005, 1)

In [9]:
def gl_sl_wrapper(alpha):
    def gl_sl(y_true, y_pred):
        return alpha*keras.losses.binary_crossentropy(y_true, y_pred) +  (1-alpha)* boundary_loss(y_true, y_pred)
    return gl_sl

In [14]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
from keras.layers import concatenate, Conv2DTranspose, Activation
from keras.layers import BatchNormalization
from keras.layers import Conv2D, Input, AvgPool2D

#convolutional block
def conv_block(x, kernelsize, filters, dropout, batchnorm=True): 
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(x)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(conv)
    if batchnorm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    return conv
"""
Standard UNet++ [Zhou et.al, 2018]
Total params: 9,041,601
"""
def unet2plus(input_shape,dropout=0, batchnorm=True):
    
    filters = [32,64, 128, 256,512]
    kernelsize = 3
    inputs = Input(input_shape, name='main_input')
    
    conv1_1 = conv_block(inputs, kernelsize, filters[0],dropout,batchnorm)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1_1)

    conv2_1=conv_block(pool1, kernelsize, filters[1],dropout,batchnorm)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2_1)

    up1_2 = Conv2DTranspose(filters[0], (2, 2), strides=(2, 2), name='up12', padding='same')(conv2_1)
    conv1_2 = concatenate([up1_2, conv1_1], name='merge12', axis=3)
    conv1_2=conv_block(conv1_2, kernelsize, filters[0],dropout,batchnorm)


    conv3_1=conv_block(pool2, kernelsize, filters[2],dropout,batchnorm)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(filters[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, conv2_1], name='merge22', axis=3)
    conv2_2=conv_block(conv2_2, kernelsize, filters[1],dropout,batchnorm)

    up1_3 = Conv2DTranspose(filters[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13', axis=3)
    conv1_3=conv_block(conv1_3, kernelsize, filters[0],dropout,batchnorm)

    conv4_1=conv_block(pool3, kernelsize, filters[3],dropout,batchnorm)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(filters[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32', axis=3)
    conv3_2=conv_block(conv3_2, kernelsize, filters[2],dropout,batchnorm)

    up2_3 = Conv2DTranspose(filters[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23', axis=3)
    conv2_3=conv_block(conv2_3, kernelsize, filters[1],dropout,batchnorm)

    up1_4 = Conv2DTranspose(filters[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14', axis=3)
    conv1_4=conv_block(conv1_4, kernelsize, filters[0],dropout,batchnorm)

    conv5_1=conv_block(pool4, kernelsize, filters[4],dropout,batchnorm)
    up4_2 = Conv2DTranspose(filters[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42', axis=3)
    conv4_2=conv_block(conv4_2, kernelsize, filters[3],dropout,batchnorm)

    up3_3 = Conv2DTranspose(filters[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33', axis=3)
    conv3_3=conv_block(conv3_3, kernelsize, filters[2],dropout,batchnorm)

    up2_4 = Conv2DTranspose(filters[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24', axis=3)
    conv2_4=conv_block(conv2_4, kernelsize, filters[1],dropout,batchnorm)

    up1_5 = Conv2DTranspose(filters[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15', axis=3)
    conv1_5=conv_block(conv1_5, kernelsize, filters[0],dropout,batchnorm)

    conv_final = layers.Conv2D(1, kernel_size=(1,1))(conv1_5)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    outputs = layers.Activation('sigmoid')(conv_final)  
    
    model = models.Model(inputs=[inputs], outputs=[outputs]) 

    return model

In [15]:
from keras import metrics
unet2= unet2plus(input_shape=(256,256,10))#binary_crossentropy
unet2.compile(optimizer = adam_v2.Adam(learning_rate = 1e-4), loss =binary_crossentropy, metrics = ['accuracy'])
unet2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 256, 256, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d_31 (Conv2D)             (None, 256, 256, 32  2912        ['main_input[0][0]']             
                                )                                                                 
                                                                                                  
 batch_normalization_31 (BatchN  (None, 256, 256, 32  128        ['conv2d_31[0][0]']              
 ormalization)                  )                                                           

In [16]:
# Train model
from keras.callbacks import ReduceLROnPlateau
reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                         factor=0.1,
                         patience=10,
                         verbose=1,
                         mode='auto',
                         min_delta=0.00003,
                         cooldown=0,
                         min_lr=0)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,verbose=1,mode='min')
save_model= ModelCheckpoint('unet2_bce.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
unet2.fit(images, masks, validation_data=(val_images,val_masks), batch_size=16, epochs=1000,verbose=1,shuffle=True,callbacks=[save_model,reduce_lr,early_stop])

Epoch 1/1000
56/56 [==============================] - ETA: 0s - loss: 0.4543 - accuracy: 0.8862
Epoch 1: val_loss improved from inf to 0.61509, saving model to unet2_bce.hdf5
56/56 [==============================] - 46s 612ms/step - loss: 0.4543 - accuracy: 0.8862 - val_loss: 0.6151 - val_accuracy: 0.6696 - lr: 1.0000e-04
Epoch 2/1000
56/56 [==============================] - ETA: 0s - loss: 0.3929 - accuracy: 0.9496
Epoch 2: val_loss improved from 0.61509 to 0.56934, saving model to unet2_bce.hdf5
56/56 [==============================] - 27s 483ms/step - loss: 0.3929 - accuracy: 0.9496 - val_loss: 0.5693 - val_accuracy: 0.7888 - lr: 1.0000e-04
Epoch 3/1000
56/56 [==============================] - ETA: 0s - loss: 0.3886 - accuracy: 0.9560
Epoch 3: val_loss improved from 0.56934 to 0.53044, saving model to unet2_bce.hdf5
56/56 [==============================] - 26s 471ms/step - loss: 0.3886 - accuracy: 0.9560 - val_loss: 0.5304 - val_accuracy: 0.8155 - lr: 1.0000e-04
Epoch 4/1000
56/56 [

In [17]:
np.save('unet2_bce-history.npy',unet2.history.history)

In [18]:
model_history = np.load('unet2_bce-history.npy', allow_pickle='TRUE').item()

In [19]:
#@title Plot accuracy and loss 
from matplotlib import pyplot as plt
## Accuracy
plt.plot(model_history['accuracy'])
plt.plot(model_history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

## Loss
plt.plot(model_history['loss'])
plt.plot(model_history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')

In [21]:
## Test images
test_dir=r'/content/gdrive/MyDrive/mudtest/'
test_images_list = os.listdir(r"{}/images/".format(test_dir))
test_masks_list = []
test_images = []
for n in test_images_list:
  test_masks_list.append(n)
  a = (np.array(rxr.open_rasterio(r"{}/images/{}".format(test_dir,n))))
  a = (a-np.min(a)) / (np.max(a)-np.min(a))
  test_images.append(a)

## Test masks
test_masks = []
for n in test_masks_list:
  a = (np.array(rxr.open_rasterio(r"{}/labels/{}".format(test_dir,n))))
  test_masks.append(a)

In [22]:
for i in range(len(test_images)):
  test_images[i] = test_images[i].astype('float32')
  test_images[i] = test_images[i].T

for i in range(len(test_masks)):
  test_masks[i] = test_masks[i].reshape(1,256,256,1)
  test_masks[i] = test_masks[i].T
for i in range(len(test_images)):
  test_images[i] = test_images[i].reshape(-1,256,256,10)

In [23]:
#@title Returns an image or array plot of mask prediction

def reconstruct_image(model, image, rounded=False):

  # Find model prediction
  reconstruction = model.predict(image).reshape(image.shape[1], image.shape[2])
  # Standardise between 0-1
  reconstruction = reconstruction/np.max(reconstruction)

  # Round to 0-1, binary pixel-by-pixel classification 
  if rounded:
    reconstruction = np.round(reconstruction)

  # Plot reconstructed mask (prediction)
  plt.imshow(reconstruction) 
'''
  Returns array of mask prediction, given model and image
'''
def reconstruct_array(model, image, rounded=False):

  # Find model prediction
  reconstruction = model.predict(image).reshape(image.shape[1], image.shape[2])

  if rounded:
    reconstruction = np.round(reconstruction)

  return reconstruction # Returns array

In [24]:
#@title Metric functions for evaluation
def accuracy_eval(model, image, mask): # Gives score of mask vs prediction
  if type(image) != list:   
    reconstruction = model.predict(image).reshape(mask.shape[1], mask.shape[2])
    reconstruction = np.round(reconstruction).flatten()
    return accuracy_score(mask.flatten(), reconstruction)

  else: # If a list of images input, find accuracy for each
    accuracy = []
    for i in range(len(image)):
      reconstruction = model.predict(image[i].reshape(1, 256, 256, 10))
      reconstruction = np.round(reconstruction).flatten()
      accuracy.append(accuracy_score(mask[i].flatten(), reconstruction))
    return accuracy

def recall_eval(model, image, mask): # Find recall score
  if type(image) != list:   
    reconstruction = model.predict(image).reshape(mask.shape[1], mask.shape[2])
    reconstruction = np.round(reconstruction).flatten()

    return recall_score(mask.flatten(), reconstruction, average='binary')

  else: # If a list of images input, find accuracy for each
    recall = []
    for i in range(len(image)):
        reconstruction = model.predict(image[i]).reshape(mask[i].shape[1], mask[i].shape[2])
        reconstruction = np.round(reconstruction).flatten()

        recall.append(recall_score(mask[i].flatten(), reconstruction, average='binary'))

    return recall

def precision_eval(model, image, mask): # Find precision score
  if type(image) != list:   
    reconstruction = model.predict(image).reshape(mask.shape[1], mask.shape[2])
    reconstruction = np.round(reconstruction).flatten()

    return precision_score(mask.flatten(), reconstruction, average='binary')

  else: # If a list of images input, find accuracy for each
    precision = []
    for i in range(len(image)):
        reconstruction = model.predict(image[i]).reshape(mask[i].shape[1], mask[i].shape[2])
        reconstruction = np.round(reconstruction).flatten()

        precision.append(precision_score(mask[i].flatten(), reconstruction, average='binary'))

    return precision

def iou_eval(model, image, mask): # Find precision score
  if type(image) != list:   
    reconstruction = model.predict(image).reshape(mask.shape[1], mask.shape[2])
    reconstruction = np.round(reconstruction).flatten()

    return jaccard_score(mask.flatten(), reconstruction, average='binary')

  else: # If a list of images input, find accuracy for each
    iou = []
    for i in range(len(image)):
        reconstruction = model.predict(image[i]).reshape(mask[i].shape[1], mask[i].shape[2])
        reconstruction = np.round(reconstruction).flatten()

        iou.append(jaccard_score(mask[i].flatten(), reconstruction, average='binary'))

    return iou

def f1_score_eval_basic(precision, recall):
    prec = np.mean(precision)
    rec = np.mean(recall)

    if prec + rec == 0:
        return 0

    return 2 * (prec * rec) / (prec + rec)

def produce_mask(image): # Outputs rounded image (binary)
  return np.round(image)


In [25]:
accuracy = (accuracy_eval(unet2, test_images, test_masks))
precision = (precision_eval(unet2, test_images, test_masks))
recall = (recall_eval(unet2, test_images, test_masks))
iou = (iou_eval(unet2, test_images, test_masks))

1/1 [==============================] - 0s 28ms/step


In [26]:
f1_score = (f1_score_eval_basic(precision, recall))

In [27]:

# Print score eval results for each model
print('model accuracy: ', np.mean(accuracy), np.std(accuracy))
# Print precision eval results for each model
print('model precision: ', np.mean(precision), np.std(precision))
# Print recall eval results for each model
print('model recall: ', np.mean(recall), np.std(recall))
# Print f1-score eval results for each model
print('model F1-score: ', np.mean(f1_score))
print('model iou: ', np.mean(iou))

model accuracy:  0.9852966079138276 0.012789758625299286
model precision:  0.9639441090051364 0.0540290313587763
model recall:  0.9682437283521315 0.053439079394282635
model F1-score:  0.966089134794228
model iou:  0.9352748686861707
